<a href="https://colab.research.google.com/github/LM1997610/NN/blob/main/NNDS_2023_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks for Data Science Applications
## Homework 1: Saliency maps for interpretability

**Name**: *Luca Mazzucco*

**Matricola**: *1997610*

> ✍ Upload the completed notebook **before 20/11/2023 at 23:59** on the Google Classroom page.

In [1]:
# need visualize.py to show images

! git clone https://github.com/LM1997610/NN

Cloning into 'NN'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 157 (delta 59), reused 39 (delta 39), pack-reused 84
Receiving objects: 100% (157/157), 2.21 MiB | 7.11 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [2]:
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
#import tensorflow_addons as tfa

import matplotlib.pyplot as plt

from NN.visualize import do_plot, show_two_images, show_images

In [3]:
# To ensure reproducible results (as much as possible)

tf.keras.utils.set_random_seed(1234)

### Overview

Neural networks are powerful tools, but they are **black-boxes**, meaning that it is difficult to provide human-understandable explanations on what they are doing. The field of **explanaibility** is concerned with finding algorithms for achieving this. In this homework, you will be guided in implementing some basic explanaibility algorithms (**saliency maps**), which is an instructive way of playing with the TensorFlow autodiff framework.

### Instructions

1. The homework is divided into four mandatory exercises (**5 points in total**), and a few optional exercises. Optional exercises are provided if you like the topic and would like to explore more; you are free to ignore them or complete as many as you want. I will not grade them but I might provide feedback for especially nice solutions.
2. Completing the homework successfully will remove 1 exercise from the end-of-term homework.
3. If your grade does not satisfy you, you are also free to complete the full EoT homework to recover it.
3. The grade can be kept for the entire academic year (up to October 2024).

**IMPORTANT - read carefully before starting**:

> 🟨 *External material*: if you use any external material or inspiration for the code, reference it *explicitly* in the corresponding cell. For the textual descriptions, copy-paste *is not allowed*. <ins>Not following these two points is an immediate 0 mark</ins>.

> 🔵 *Grammar*: for the textual descriptions, I will remove points for too many grammatical or textual errors. Please try to be precise and provide nice-to-read descriptions, like if you were writing a report.

> 🟥 *Vectorization and TensorFlow*: the homework must be done _fully in TensorFlow_ and vectorizing the code as much as possible (e.g., do not loop explicitly over the batch dimension).

> 🟪 *Math*: you can also use LaTeX in Markdown if you need to write equations or if you need generic math notation.

## Warmup: Data loading

For this homework, you can select any **tabular dataset** that you like, for either classification or regression. A few repositories that you can look at:

1. The catalog of [TensorFlow Datasets](https://www.tensorflow.org/datasets/).
2. The [Kaggle catalog](https://www.kaggle.com/data). For downloading data from Kaggle on Google Colab, you will need to [load your Kaggle authentication token](https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb).
3. The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php).
4. The [🤗 HuggingFace Datasets](https://huggingface.co/docs/datasets/) repository.

You are not bound to these; any open repository is okay. The choice of dataset will not influence the mark.

✍ **DESCRIPTION OF THE CODE**

*Provide a small description of the dataset below (e.g., source, task, bibliographic reference if necessary...), both as text and in the comments of the code.*

**TODO**: add description here (1-2 paragraphs).

In [4]:
# TODO: Insert any data loading code here. If the data loading part is complex,
# consider using a separate .py file that you can attach to the submission.

(train_data, val_data, test_data), data_info = tfds.load("cifar10",
                                                        split=["train[0:98%]", "train[98%:]", "test"],
                                                        batch_size = 200,
                                                        as_supervised = True,
                                                        with_info = True,
                                                        shuffle_files = True)

def preproccess(image, label):
  image = tf.cast(image, tf.float32)/ 255.0
  return image, label

train_data = train_data.map(lambda image, label: preproccess(image, label))
val_data = val_data.map(lambda image, label: preproccess(image, label))
test_data = test_data.map(lambda image, label: preproccess(image, label))

# X, y = ...

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete2HAFCF/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incomplete2HAFCF/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


The dataset chosen is **CIFAR-10**, a standard dataset for image classification, widely used as a benchmark across various models.\
Dataset  created by [Canadian Institute For Advanced Research](https://en.wikipedia.org/wiki/Canadian_Institute_for_Advanced_Research), published in 2009, here loaded from TensorFlow Dataset.\
Additional information follows :

In [5]:
# description of the dataset:

print("\nDescription of the dataset:\n")
print(f"Dataset name : {data_info.name}")
print(data_info.description, "\n")

print(f"Image shape: {data_info.features['image'].shape} \n")
print(f"Class names: {data_info.features['label'].names} \n")

print(f"train_dataset:  {sum(len(x[0]) for x in train_data.as_numpy_iterator())} samples")
print(f"val_dataset:  {sum(len(x[0]) for x in val_data.as_numpy_iterator())} samples")
print(f"test_dataset:  {sum(len(x[0]) for x in test_data.as_numpy_iterator())} samples")


Description of the dataset:

Dataset name : cifar10
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

Image shape: (32, 32, 3) 

Class names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

train_dataset:  49000 samples
val_dataset:  1000 samples
test_dataset:  10000 samples


### Exercise 1: Train a neural network model (1 point)

Define, train, and test a neural network for the provided dataset.

☝ You are free to make any modelling choice (e.g., activation function, normalization layers, etc.), provided the result makes sense.

✅ **Completion requirement**: print on screen the test accuracy of the network. Additional comments and visualizations are also appreciated.

In [6]:
def add_conv_block(model, n_filters):

  model.add(tf.keras.layers.Conv2D(filters = n_filters,                 # conv_layer
                                kernel_size = (3, 3),
                                padding = "same"))

  #model.add(tf.keras.layers.Conv2D(filters = n_filters,                 # conv_layer
  #                              kernel_size = (3, 3),
  #                              padding = "same"))

  model.add(tf.keras.layers.BatchNormalization())                       # batch_normalization
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))  # pool_layer
  model.add(tf.keras.layers.Activation('relu'))                         # activation
  model.add(tf.keras.layers.Dropout(0.5))                               # drop_out

  return model

In [7]:
# TODO: define a suitable neural network.

hidden_layers =  [128, 512, 512, 512, 512]

model = tf.keras.Sequential()

for i in hidden_layers:
  model = add_conv_block(model, i)

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(10))

In [8]:
# TODO: train the neural network.

lr = 0.001
weight_decay = 0.001
n_epoches = 30
milestones = list(range(1, n_epoches))
gamma = 0.90

cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
optimizer = tf.keras.optimizers.Adam(learning_rate = lr, weight_decay = weight_decay)

def scheduler(epoch, lr):
    if epoch in milestones:
        lr *= gamma
    return lr

call= [tf.keras.callbacks.TerminateOnNaN(),
      tf.keras.callbacks.LearningRateScheduler(scheduler)]
      #tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=2)]

model.compile(optimizer= optimizer, loss = cross_entropy, metrics = accuracy)

history = model.fit(train_data, epochs=n_epoches, validation_data = val_data, callbacks = call, verbose=2)

Epoch 1/30


KeyboardInterrupt: ignored

In [ ]:
# TODO: test the neural network and print the result on screen.

do_plot(history, len(history.history["loss"]), 3)

test_loss, test_acc = model.evaluate(test_data, verbose=1); print()

print(f" >> test Loss: {round(test_loss, 4)}")
print(f" >> test Accuracy : {round(test_acc, 4)}")

### Exercise 2: Computing a vanilla saliency map (1 points)

> Before starting, I suggest you read [1] as a warm-up. This is one of the first papers that tried to apply this kind of techniques to modern neural networks.

What do we mean by explainability? Consider the neural network $f(\cdot)$ you just trained, and a prediction $\hat{y} = f(x)$ we want to analyze. **Feature attribution** methods try to assign a weight $w_i$ to each input feature $x_i$, to understand which parts of the input have contributed the most to the explanation.

The simplest feature attribution technique, called **vanilla saliency map**, simply computes the gradient at that point:

$$
  S(x) = \left\lvert \frac{\partial f_c(x)}{\partial x} \right\rvert
$$

where  $c$ is the index corresponding to the predicted class.

✅ **Completion requirement**: Take any point from your test dataset, and compute a saliency map using `tf.GradientTape`. Check the weight to see if you can find anything to "interpret". **Note**: I am not evaluating how nice / good the explanation is, only the code.

In [ ]:
# TODO: Take an element from your test set and compute the saliency map

# take first batch from test dataset
test_images, test_labels = zip(*[(image, label) for image, label in test_data])
test_images, test_labels = test_images[0], test_labels[0]

def saliency_map(index):

  with tf.GradientTape() as tape:

    tensor_input = tf.expand_dims(test_images[index], axis=0) # add batch dimension to tensor image
    tape.watch(tensor_input)                                  # keep track of tensor_input in gradient computation

    class_scores = model(tensor_input, training=False)        # get preticions from the model
    predicted_class = np.argmax(class_scores)                 # predicted class index

    class_channel = class_scores[:, predicted_class]          # predicted class logit (before softmax)

  grads = tape.gradient(class_channel, tensor_input)          # compute gradient of class_channel with respect tensor_input

  # valuable reference used for pixel normalization this link :
  # https://usmanr149.github.io/urmlblog/cnn/2020/05/01/Salincy-Maps.html

  dgrad_abs = tf.math.abs(grads)
  dgrad_max = np.max(dgrad_abs, axis=-1)[0]

  # normalize pixel in range 0-1
  arr_min, arr_max  = np.min(dgrad_max), np.max(dgrad_max)
  map = (dgrad_max - arr_min) / (arr_max - arr_min + 1e-18)

  return map, grads

In [ ]:
# TODO: Check the saliency map to analyze the result. What can you say about the map?

img_indexes = [34, 41]    # index of test images to show

for i in img_indexes:
  map, grad = saliency_map(i)
  show_two_images(test_images[i], map)

Given an input image $x$, the **Saliency map** shows how a slight variation in the pixels of the image influences the classification score for the label of $x$. This highlights the pixels that the model considered to be most relevant in determining the class of $x$.

The results obtained are positive: in the saliency maps is possible to distinguish the position of the original subject in the image, but the actual shape of the subject is masked by some noise

### Exercise 3: Advanced saliency maps (1 point)

> For this exercise, you can read [2] for an overview on the limits of vanilla saliency maps and a description of SmoothGrad.

Saliency maps have several issues: most notably, they suffer from noise and they are not stable to small changes in the input or in the model (try running again the training and interpreting the same point). Many methods have been proposed to overcome this.

**[SmoothGrad](https://arxiv.org/abs/1706.03825)**, for example, computes multiple saliency maps from noisy versions of the input:

$$
  \text{SmoothGrad}(x) = \frac{1}{n}\sum_{i=1}^n S(x + \varepsilon_i), \;\; \varepsilon_i \sim \mathcal{N}(0, \sigma^2I)
$$

where $\varepsilon$ is a vector of the same shape as $x$, whose values are sampled from a normal distribution with zero mean and small variance.

🟩 **Completion requirement**: Implement the SmoothGrad procedure for the same point. Has the explanation improved? Bonus points if you can avoid running a for-loop, and by calling the gradient operation a single time.


In [ ]:
def SmoothGrad(index, N = 100, sd = 0.0):

  e = tf.random.normal(shape=(N, 32, 32, 3), mean = 0.0, stddev = sd)

  with tf.GradientTape() as tape:

    tensor_input = test_images[index] + e                # add tensor of noise to input image: shape -> (N, 32, 32, 3)
    tape.watch(tensor_input)                             # keep track of tensor_input in gradient computation
    class_scores = model(tensor_input, training=False)   # get preticions from the model
    predicted_class = np.argmax(class_scores, axis =1)   # predicted class index (N,)

    indices = tf.stack([tf.range(N), predicted_class], axis=-1)
    class_channel = tf.gather_nd(class_scores, indices)  # returns logits from class_scores corresponding to indices predicted

  grads = tape.gradient(class_channel, tensor_input)     # shape -> (N, 32, 32, 3)
  mean_tensor = tf.reduce_mean(grads, axis=0)            # mean over the batch dimension

  grads = tf.math.abs(mean_tensor)
  grads = np.max(grads, axis=-1)

  return grads

In [ ]:
def evaluate_on_sigma(index, noise_list = [0.0, 0.10, 0.20, 0.30]):

    map_list = []
    map_list.append(test_images[index])

    titles = ["noise_level :"] + [str(x)+' %' for x in noise_list]

    for noise_level in noise_list:

        # not actually needed: images pixel already normalized in range 0-1 -> sigma = noise level
        sigma = noise_level*(tf.reduce_max(test_images[index]) - tf.reduce_min(test_images[index]))

        this_map = SmoothGrad(index, N = 100, sd = sigma)
        map_list.append(this_map)

    return map_list, titles

In [ ]:
# TODO: Take an element from your test set and compute SmoothGrad.
# Check the results and compare with respect to the previous exercise.

images_to_show = [4, 26, 50]
show_images(images_to_show, evaluate_on_sigma)

Comparing Vanilla gradient maps (0% noise) with Smoothed ones, the latter seem to be more valid and they provide better explanations.\
Pixels identified as most significant closely mirror pixels from the original image.

### Exercise 4: Global explanations (2 points)

The previous exercises are examples of **local** explanations, where we try to interpret a single prediction of the network. Sometimes we are interested in **global** explanations, that try to find common patterns of behaviour. Suppose we have a dataset $\mathcal{T} = \left\{x_i\right\}$ of examples, we can compute some approximate global measure of influence by averaging their saliency:

$$
\text{GlobalSaliency} = \frac{1}{n} \sum_i S(x_i)
$$

To make this exercise more interesting, we will split it into 3 parts.

**Exercise 4.1**: write a function to compute in parallel the saliency for multiple examples. Note that the resulting matrix $S$ will have shape $(n, d)$, where $n$ is the number of examples and $d$ the size of the input, which is the Jacobian of the network. Try to write the function by avoiding for-loops and multiple tapes, using the [proper tools from TensorFlow](https://www.tensorflow.org/guide/advanced_autodiff).

In [ ]:
# TODO: Write the required function, possibly avoding for-loops.

def GlobalSaliency(test_dataset):

  with tf.GradientTape(persistent=True) as tape:

    inputs = tf.convert_to_tensor(test_dataset)            # input tensor is first Test batch: shape -> (N, 32, 32, 3)
    tape.watch(inputs)                                     # keep track of input in gradient computation
    class_scores = model(inputs, training=False)           # get preticions from the model
    predicted_class = np.argmax(class_scores, axis =1)     # predicted class index (N,)

    indices = tf.stack([tf.range(test_dataset.shape[0]), predicted_class], axis=-1)
    class_channel = tf.gather_nd(class_scores, indices)    # returns logits from class_scores corresponding to indices predicted

  grads = tape.gradient(class_channel, inputs)   # compute gradient of class_channel with respect tensor_input, shape -> (N, 32, 32, 3)
  mean_tensor = tf.reduce_mean(grads, axis=0)    # mean over the batch dimension

  grads = tf.math.abs(mean_tensor)
  grads = np.max(grads, axis=-1)

  ## normalize to range between 0 and 1
  arr_min, arr_max  = np.min(grads), np.max(grads)
  map = (grads - arr_min) / (arr_max - arr_min + 1e-18)

  return map

**Exercise 4.2**: write a function to compute the global saliency and try to explain the results.

In [ ]:
# TODO: Write the required function.

global_map = GlobalSaliency(test_images)

plt.figure(figsize=(3, 3))
plt.title("\n Global Saliency Map \n")
im = plt.imshow(global_map)
plt.axis("off")

cbar = plt.colorbar(im, fraction=0.046, pad=0.09)
plt.show()

From the Global Saliency map, central pixels of images appear to wield significant influence in label assignment.\
This makes sense since subjects are usually placed in the center of the images.

**Exercise 4.3**: note that a linear model $f(x) = w^\top x + b$ is an example of an *intrinsically* interpretable  model, since the weights $w$ can be checked to analyze the global saliency of each feature (see [3]).

In [ ]:
# TODO: Compare the results you obtained before with a simpler linear model.

linear_model = tf.keras.Sequential()
linear_model.add(tf.keras.layers.Flatten())
linear_model.add(tf.keras.layers.Dense(10))  # FeedForward Network with a single layer

# Compile the model
linear_model.compile(optimizer="adam", loss = cross_entropy, metrics= accuracy )

call_backs = [tf.keras.callbacks.TerminateOnNaN(),
              tf.keras.callbacks.LearningRateScheduler(scheduler),
              tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True, verbose = 2),]
              #tfa.callbacks.TQDMProgressBar(leave_epoch_progress = False, show_epoch_progress = False)]

# Train the model
linear_history = linear_model.fit(train_data, epochs = 50, validation_data = val_data, callbacks = call_backs , verbose=0)
do_plot(linear_history, len(linear_history.history["loss"]), 4)

# Evaluate the model
test_loss_linear, test_acc_linear = linear_model.evaluate(test_data, verbose=1); print()

print(f" >> test Loss: {round(test_loss_linear, 4)}")
print(f" >> test Accuracy : {round(test_acc_linear, 4)}")

In [ ]:
w = linear_model.get_weights()[0]

w = tf.reshape(w, (10, 32, 32, 3))

mean_w = tf.reduce_mean(w, axis=0)
w_abs = tf.math.abs(mean_w)
w_max = np.max(w_abs, axis=-1)

## normalize to range between 0 and 1
arr_min, arr_max  = np.min(w_max), np.max(w_max)
linear_map = (w_max - arr_min) / (arr_max - arr_min + 1e-18)


# ------------- Show maps -----------------
fig, ax = plt.subplots(1, 2, figsize=(7, 7))

ax[0].imshow(global_map, aspect="equal")
im = ax[1].imshow(linear_map, aspect="equal")

ax[0].set_title("\n Global Saliency Map \n ConvNet \n")
ax[1].set_title("\n Global Saliency Map \n Linear \n")
ax[0].axis('off'); ax[1].axis('off')

cax = fig.add_axes([0.92, 0.3, 0.02, 0.39])
fig.colorbar(im, cax, orientation='vertical', extend="both")

plt.subplots_adjust(wspace=0.5, right = 0.85)
plt.show()

### Optional exercises and parting words

> ⚠ Explainability is a complex topic, with multiple issues arising from the over-abundance of techniques, their instability, etc. While an interesting research topic to pursue, never use blindly these techniques in high-stake applications!

These exercises were just a brief and short introduction to the topic of explainability. Below you can find some additional exercises to tackle if you are interested. Remember that these are not part of your grade, but I am happy to provide feedback if they are of interest to you.

1. There are dozens of possible variations on feature attribution methods, which may or may not provide better results (see [4] for a benchmarking and this nice [Distill blog post](https://distill.pub/2020/attribution-baselines/)). **[Integrated Gradients](https://arxiv.org/abs/1703.01365)** are an interesting example, where the saliency is integrated over a path ranging from an empty input to the true input. Try implementing integrated gradients.
2. **Data attribution** methods are a different class of explanation methods, which try to predict what points in the dataset where most influential to a given prediction (e.g., a picture of a cat will be especially influential on similar pictures). One example of such methods is TracIn [5], which stores checkpoints of the model during training and evaluates the correlation of the gradients. Try to implement TracIn or any other metric of data influence.
3. A recent line of research tries to use large language models (e.g., ChatGPT) to explain other models (e.g., see [Language models can explain neurons in language models](https://openai.com/research/language-models-can-explain-neurons-in-language-models)). If you have access to an LLM, you can try it! Take a specific neuron in the model, and collect the activation for multiple examples. Provide these activations to the LLM, and prompt it to provide a human-understandable explanation. What is the result?

### Final checklist

1. Carefully check all code. Insert comments when needed. Search for "TODO" to see if you forgot something.
2. Run everything one final time. *Please do not send me notebooks with errors or cells that are not working.*
3. Upload the completed notebook **before 20/11/2023 23:59** on the Google Classrom page.

### Bibliography

[1] Simonyan, K., Vedaldi, A. and Zisserman, A., 2013. [Deep inside convolutional networks: Visualising image classification models and saliency maps](https://arxiv.org/abs/1312.6034). arXiv preprint arXiv:1312.6034.

[2] Smilkov, D., Thorat, N., Kim, B., Viégas, F. and Wattenberg, M., 2017. [SmoothGrad: removing noise by adding noise](https://arxiv.org/abs/1706.03825). arXiv preprint arXiv:1706.03825.

[3] Rudin, C., 2019. [Stop explaining black box machine learning models for high stakes decisions and use interpretable models instead](https://www.nature.com/articles/s42256-019-0048-x). Nature Machine Intelligence, 1(5), pp. 206-215.

[4] Nguyen, G., Kim, D. and Nguyen, A., 2021. [The effectiveness of feature attribution methods and its correlation with automatic evaluation scores](https://proceedings.neurips.cc/paper/2021/hash/de043a5e421240eb846da8effe472ff1-Abstract.html). Advances in Neural Information Processing Systems, 34, pp.26422-26436.

[5] Pruthi, G., Liu, F., Kale, S. and Sundararajan, M., 2020. [Estimating training data influence by tracing gradient descent](https://proceedings.neurips.cc/paper/2020/hash/e6385d39ec9394f2f3a354d9d2b88eec-Abstract.html). Advances in Neural Information Processing Systems, 33, pp. 19920-19930.